# Grouping tasks

In this section we are going to see how we can group tasks. Grouping tasks might be useful for various reasons, for instance, for reusability purposes (as seen in a previous section) or even just for readability purposes. 

We will also see very briefly how to use the functions [functions.text.print_title](../../ref/functions/text.rst#brigade.plugins.functions.text.print_title) and [functions.text.print_result](../../ref/functions/text.rst#brigade.plugins.functions.text.print_result) to make things look pretty.

As an objective in this tutorial we are going to be configuring the hostname and domain name of our network devices.

Let's start with the basic imports and objects we will need:

In [1]:
from brigade.core import InitBrigade
from brigade.plugins.tasks import networking, text
from brigade.plugins.functions.text import print_title, print_result

brg = InitBrigade(config_file="config.yaml", dry_run=True)
cmh = brg.filter(site="cmh", type="network_device")

You may have noticed that `InitBrigade` got a new argument `dry_run=True`. That argument (which defaults to `False`) controls whether to apply the changes to the device or just simulate them. You can control that argument via the configuration as well. You can even override it in a given task if you want (we will see an example later on).

Now, let's create task that is going to group what we want to do:

In [2]:
def basic_configuration(task):
    # Transform inventory data to configuration via a template file
    r = task.run(text.template_file,
                 name="Base Configuration",
                 template="base.j2",
                 path=f"templates/{task.host.nos}")
    
    # Save the compiled configuration into a host variable
    task.host["config"] = r.result
    
    # Deploy that configuration to the device using NAPALM
    task.run(networking.napalm_configure,
             name="Loading Configuration on the device",
             replace=False,
             configuration=task.host["config"])

As you can see the task is basically doing two things:

1. Render configuration from a jinja2 template and storing it into a host variable. Note we are using the host network operating system as part of the path so we can load the right template for the given device platform.
2. Deploying that configuration to the device with NAPALM.

Quite straightforward. For reference let's look at the templates:

In [3]:
%cat templates/eos/base.j2

hostname {{ host }}
ip domain-name {{ site }}.{{ domain }}


In [4]:
%cat templates/junos/base.j2

system {
  host-name {{ host }};
  domain-name {{ site }}.{{ domain }};
}


Now we can proceed with just a few lines of code: 

In [5]:
print_title("Playbook to configure the network")
result = cmh.run(basic_configuration)
print_result(result)

**** Playbook to configure the network *****************************************
basic_configuration*************************************************************
* spine00.cmh ** changed : True ************************************************
---- basic_configuration ** changed : False  -----------------------------------
---- Base Configuration ** changed : False  ------------------------------------
hostname spine00.cmh
ip domain-name cmh.acme.local
---- Loading Configuration on the device ** changed : True  --------------------
@@ -7,6 +7,9 @@
    action bash sudo /mnt/flash/initialize_ma1.sh
 !
 transceiver qsfp default-mode 4x10G
+!
+hostname spine00.cmh
+ip domain-name cmh.acme.local
 !
 spanning-tree mode mstp
 !

* spine01.cmh ** changed : True ************************************************
---- basic_configuration ** changed : False  -----------------------------------
---- Base Configuration ** changed : False  ------------------------------------
system {
  host-name spine

As you can see, pretty straightforward. We just call the function we defined earlier grouping the tasks we wanted to execute and then we leverage on `functions.text.print_title` and `tasks.text.print_result` to make things look pretty.

## Overriding `dry_run` per task

This is not strictly related to task groups but as the concept of `dry_run` was introduced here let's see now how we can override the behavior of `dry_run` per task. By default, tasks are going to honor the value of the attribute `dry_run` of our Brigade object which is defined when creating the object:

In [6]:
cmh.dry_run

True

We can change that value globally by just doing `cmh.dry_run = False` or we can override if needed just for a given task by passing the desired value as a keyword argument. For instance:

In [7]:
print_title("Playbook to configure the network")
result = cmh.run(basic_configuration, dry_run=False)
print_result(result)

**** Playbook to configure the network *****************************************
basic_configuration*************************************************************
* spine00.cmh ** changed : True ************************************************
---- basic_configuration ** changed : False  -----------------------------------
---- Base Configuration ** changed : False  ------------------------------------
hostname spine00.cmh
ip domain-name cmh.acme.local
---- Loading Configuration on the device ** changed : True  --------------------
@@ -7,6 +7,9 @@
    action bash sudo /mnt/flash/initialize_ma1.sh
 !
 transceiver qsfp default-mode 4x10G
+!
+hostname spine00.cmh
+ip domain-name cmh.acme.local
 !
 spanning-tree mode mstp
 !

* spine01.cmh ** changed : True ************************************************
---- basic_configuration ** changed : False  -----------------------------------
---- Base Configuration ** changed : False  ------------------------------------
system {
  host-name spine

This time, configuration should've been commited so if we run it again the task "Loading Configuration on the device" should tell us that `changed : False` and should return an empty diff. Let's see if that's true:

In [8]:
print_title("Playbook to configure the network")
result = cmh.run(basic_configuration, dry_run=False)
print_result(result)

**** Playbook to configure the network *****************************************
basic_configuration*************************************************************
* spine00.cmh ** changed : False ***********************************************
---- basic_configuration ** changed : False  -----------------------------------
---- Base Configuration ** changed : False  ------------------------------------
hostname spine00.cmh
ip domain-name cmh.acme.local
---- Loading Configuration on the device ** changed : False  -------------------

* spine01.cmh ** changed : False ***********************************************
---- basic_configuration ** changed : False  -----------------------------------
---- Base Configuration ** changed : False  ------------------------------------
system {
  host-name spine01.cmh;
  domain-name cmh.acme.local;
}
---- Loading Configuration on the device ** changed : False  -------------------

* leaf00.cmh ** changed : False ***************************************